In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import math
import tensorflow as tf
from scipy.sparse import csr_matrix
from fastpivot import pivot_table, pivot_sparse
from sklearn.metrics import pairwise_distances
from surprise import KNNWithMeans
from scipy.sparse import csr_matrix, issparse 
from scipy.stats import pearsonr

In [29]:
# movies = pd.read_csv('../datasets/movielens_original/movies.csv')
movies = pd.read_csv('../datasets/ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [30]:
# ratings = pd.read_csv('../datasets/movielens_original/ratings.csv')
ratings = pd.read_csv('../datasets/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [31]:
df = ratings.merge(movies, how = 'inner', on='movieId',)
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [32]:
model = tf.keras.models.load_model('saved_model.h5')

In [33]:
movies_ids = list(set(list(df.movieId.unique())))
users_ids = list(set(list(df.userId.unique())))

dict_movies = {}
index = 0

for ids in sorted(movies_ids):
    dict_movies[ids] = index
    index += 1

dict_users = {}
index = 0
for ids in sorted(users_ids):
    dict_users[ids] = index
    index += 1


In [55]:
user = 1
predicted_rating = model.predict([pd.Series([user_id]*len(dict_movies)), pd.Series(dict_movies.values())])
predicted_rating = pd.DataFrame(predicted_rating)
predicted_rating

304/304 [==============================] - 1s 3ms/step


,0
0,2.932035
1,3.444300
2,3.228229
3,3.583076
4,3.183743
...,...
9719,3.333295
9720,3.949676
9721,3.237458
9722,3.600155


In [56]:
matrix = df.pivot_table(index='userId', columns='title', values = 'rating')

matrix_norm = matrix.subtract(matrix.mean(axis=1), axis = 'rows')
user_similarity = matrix_norm.T.corr()
user_similarity.drop(index=user, inplace=True)
user_similarity.head()

user_similarity_cosine = cosine_similarity(matrix_norm.fillna(0))
user_similarity_cosine

n = 50
user_similarity_threshold = 0.3

similar_users = user_similarity[user_similarity[user]>user_similarity_threshold][user].sort_values(ascending=False)[:n]
similar_users

userId
598    1.000000
333    1.000000
146    1.000000
106    1.000000
550    1.000000
473    0.962250
511    0.925820
9      0.918559
13     0.878310
366    0.872872
401    0.866921
535    0.866400
154    0.866025
481    0.866025
90     0.821584
499    0.802955
157    0.801784
139    0.790569
476    0.786936
487    0.774597
210    0.767649
114    0.759257
530    0.755929
49     0.750000
162    0.708333
297    0.706281
207    0.701646
44     0.684448
430    0.654654
394    0.650600
574    0.643796
421    0.641223
248    0.624695
173    0.617213
65     0.612372
505    0.612372
369    0.612098
435    0.610960
375    0.606128
431    0.603023
351    0.600000
72     0.596432
206    0.596285
120    0.594874
344    0.587609
112    0.583892
99     0.583450
445    0.569069
494    0.567350
467    0.566947
Name: 1, dtype: float64

WIlson

In [35]:
def create_X(df): 
    """ 
    Generates a sparse matrix from ratings dataframe. 
        
    Args: 
        df: pandas dataframe 
        
    Returns: 
        X: sparse matrix 
        user_mapper: dict that maps user id's to user indices 
        user_inv_mapper: dict that maps user indices to user id's 
        movie_mapper: dict that maps movie id's to movie indices 
        movie_inv_mapper: dict that maps movie indices to movie id's 
    """ 
    N = ratings['userId'].nunique() 
    M = ratings['movieId'].nunique() 

    user_mapper = dict(zip(np.unique(ratings["userId"]), list(range(N)))) 
    movie_mapper = dict(zip(np.unique(ratings["movieId"]), list(range(M)))) 
        
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"]))) 
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"]))) 
        
    user_index = [user_mapper[i] for i in df['userId']] 
    movie_index = [movie_mapper[i] for i in df['movieId']] 

    # X = csr_matrix((ratings["rating"], (user_index, movie_index)), shape=(N, M)) 
    X = csr_matrix((ratings["rating"], (movie_index, user_index)), shape=(M, N)) 
        
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

In [36]:
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(ratings)

In [37]:
user_movie_matrix = pd.DataFrame.sparse.from_spmatrix(X)
user_movie_matrix

,0,1,2,3,4,5,6,7,8,9,...,600,601,602,603,604,605,606,607,608,609
0,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
1,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
user = 585

# def user_based_recom(input_dataframe,input_user_id):    
#     pivot_user_based = pd.pivot_table(input_dataframe, index='title', columns=['userId'], values='rating').T
#     sparse_pivot_ub = sparse.csr_matrix(pivot_user_based.fillna(0))
#     user_recomm = pw.cosine_similarity(sparse_pivot_ub)
#     user_recomm_df = pd.DataFrame(user_recomm,columns=pivot_user_based.index.values,
#                  index=pivot_user_based.index.values)
#     ## Item Rating Based Cosine Similarity
#     usr_cosine_df = pd.DataFrame(user_recomm_df[input_user_id].sort_values(ascending=False))
#     usr_cosine_df.reset_index(level=0, inplace=True)
#     usr_cosine_df.columns = ['userId','cosine_sim']
#     return usr_cosine_df

# similarity = user_movie_matrix.corrwith(user_movie_matrix[user-1])
# similarity

# similarity = pd.DataFrame()
# column = []
# corrs = []
# p_values = []

# for n in user_movie_matrix:
#     if (user - 1 != n):
#         column.append(n) #Butuh tambah 1
#         corr, p_value = pearsonr(user_movie_matrix[user-1], user_movie_matrix[1])
#         corrs.append(corr)
#         p_values.append(p_value)
# similarity['user'] = column
# similarity['corr'] = corrs
# similarity['p_value'] = p_values
# similarity

# user_similarity = user_movie_matrix.iloc[user-1].T.corr(user_movie_matrix)

user_similarity_cosine = cosine_similarity(user_movie_matrix, user_movie_matrix.iloc[user-1])
user_similarity_cosine

ValueError: Expected 2D array, got 1D array instead:
array=[0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 2.  0.  3.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  3.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  4.5 0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  4.  0.  0.  0.  0.  0.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  3.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  

,0,1,2,3,4,5,6,7,8,9,...,162531,162532,162533,162534,162535,162536,162537,162538,162539,162540
0,0.0,3.5,4.0,3.0,4.0,0.0,0.0,4.0,0.0,3.5,...,0.0,4.5,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
X.T.corr()

AttributeError: corr not found

In [65]:
user_similarity = user_movie_matrix.T.corr()

KeyboardInterrupt: 

In [74]:
user_movie_matrix.loc[0]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
59042    0.0
59043    0.0
59044    0.0
59045    0.0
59046    0.0
Name: 0, Length: 59047, dtype: Sparse[float64, 0]

In [63]:
movies[movies.movieId==1]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [49]:
df[df.movieId == 1]

,userId,movieId,rating,timestamp,title,genres
601190,2,1,3.5,1141415820,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
601191,3,1,4.0,1439472215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
601192,4,1,3.0,1573944252,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
601193,5,1,4.0,858625949,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
601194,8,1,4.0,890492517,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
658494,162529,1,2.0,888181499,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
658495,162530,1,5.0,989808332,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
658496,162533,1,4.5,1280920369,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
658497,162534,1,4.0,1526714137,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [36]:
test = df.groupby('userId').movieId.count()
user_ids = test[test > 20].index
df[df.userId.isin(user_ids)].movieId.nunique()

59044

In [34]:
df[df.groupby('userId').count() > 20]

,userId,movieId,rating,timestamp,title,genres
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,296.0,5.0,1.439474e+09,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
2,NaN,296.0,4.0,1.573939e+09,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3,NaN,296.0,4.0,8.307862e+08,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4,NaN,296.0,4.0,8.354447e+08,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
...,...,...,...,...,...,...
25000090,NaN,NaN,NaN,NaN,NaN,NaN
25000091,NaN,NaN,NaN,NaN,NaN,NaN
25000092,NaN,NaN,NaN,NaN,NaN,NaN
25000093,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
n_users = df.userId.unique().shape[0]
n_movies = df.userId.unique().shape[0]
matrix = np.zeros((n_users, n_movies))
for line in ratings.itertuples():
    matrix[line[1]-1, line[2]-1] = line[3]

IndexError: index 164178 is out of bounds for axis 1 with size 162541

In [24]:
user = df[df.userId == 585]
user = csr_matrix((user['rating'].astype(int), (user['userId'].astype(int)-1, user['movieId'].astype(int)-1)))

In [21]:
correlation = df.T.corr()
correlation.head()

/var/folders/fm/r45pd3w141dg56rt5cl8mht80000gn/T/ipykernel_2086/1174692981.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = df.T.corr()


""


In [27]:
user.shape

(585, 111362)

In [28]:
user_movie_ratings.shape

(162541, 209171)

In [29]:
user_ratings = ratings[ratings['userId'] == user_id]

user_movie_ratings = csr_matrix((df['rating'].astype(int), (df['userId'].astype(int)-1, df['movieId'].astype(int)-1)))
# user_movie_ratings.T.corr()

cosine_similarities = pairwise_distances(user_movie_ratings, user, metric='cosine')
# cosine_similarities = cosine_similarity(user_movie_ratings.T, user.T)

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 209171 while Y.shape[1] == 111362

In [3]:
# create sparse matrix from DataFrame
matrix_sparse = csr_matrix((df['rating'].astype(int), (df['userId'].astype(int), df['movieId'].astype(int))))

# calculate row means
row_means = matrix_sparse.mean(axis=1)

# create sparse matrix of row means
row_means_matrix = csr_matrix(row_means)

# normalize matrix by subtracting row means
matrix_norm = matrix_sparse - row_means_matrix

# calculate cosine similarity between users
user_similarity_cosine = cosine_similarity(matrix_norm.T.fillna(0))

NameError: name 'df' is not defined

In [8]:
pivot_fast

pivot_col,col0,col1,col10,col100,col101,col102,col103,col104,col105,col106,...,col990,col991,col992,col993,col994,col995,col996,col997,col998,col999
pivot_idx,,,,,,,,,,,,,,,,,,,,,
idx0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
idx1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
idx10,0.0,-0.336366,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
idx100,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
idx101,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,...,2.363482,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
idx995,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,...,0.000000,1.424630,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
idx996,0.0,0.000000,0.000000,0.0,0.0,0.0,0.152594,0.000000,0.00000,2.597955,...,0.000000,0.000000,0.000000,0.0,-0.437121,1.09762,0.000000,0.0,0.0,0.0
idx997,0.0,1.087749,0.000000,0.0,0.0,0.0,0.000000,0.082645,-0.29158,0.000000,...,0.000000,0.525875,0.128258,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
